# Zadanie 1

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, udf
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DateType
import locale

spark = SparkSession.builder \
    .appName("Zadanie_1") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/22 01:20:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
schema = StructType([
    StructField("Kraj", StringType(), True),
    StructField("Sprzedawca", StringType(), True),
    StructField("Data zamowienia", StringType(), True),
    StructField("idZamowienia", IntegerType(), True),
    StructField("Utarg", StringType(), True),
])

In [3]:
file_path = "zamowienia.txt"
df = spark.read.option("delimiter", ";") \
               .option("header", True) \
               .csv(file_path)
df.show()

+------+----------+---------------+------------+-----------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia|      Utarg|
+------+----------+---------------+------------+-----------+
|Polska|  Kowalski|     16.07.2003|       10248|  440,00 z|
|Polska|  Sowiäski|     10.07.2003|       10249|1 863,40 z|
|Niemcy|   Peacock|     12.07.2003|       10250|1 552,60 z|
|Niemcy| Leverling|     15.07.2003|       10251|  654,06 z|
|Niemcy|   Peacock|     11.07.2003|       10252|3 597,90 z|
|Niemcy| Leverling|     16.07.2003|       10253|1 444,80 z|
|Polska|  Kowalski|     23.07.2003|       10254|  556,62 z|
|Polska|     Dudek|     15.07.2003|       10255|2 490,50 z|
|Niemcy| Leverling|     17.07.2003|       10256|  517,80 z|
|Niemcy|   Peacock|     22.07.2003|       10257|1 119,90 z|
|Niemcy|   Davolio|     23.07.2003|       10258|1 614,88 z|
|Niemcy|   Peacock|     25.07.2003|       10259|  100,80 z|
|Niemcy|   Peacock|     29.07.2003|       10260|1 504,65 z|
|Niemcy|   Peacock|     

In [4]:
df = df.withColumn("Sprzedawca", regexp_replace(col("Sprzedawca"), "ä", "ń"))

df.show()

+------+----------+---------------+------------+-----------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia|      Utarg|
+------+----------+---------------+------------+-----------+
|Polska|  Kowalski|     16.07.2003|       10248|  440,00 z|
|Polska|  Sowiński|     10.07.2003|       10249|1 863,40 z|
|Niemcy|   Peacock|     12.07.2003|       10250|1 552,60 z|
|Niemcy| Leverling|     15.07.2003|       10251|  654,06 z|
|Niemcy|   Peacock|     11.07.2003|       10252|3 597,90 z|
|Niemcy| Leverling|     16.07.2003|       10253|1 444,80 z|
|Polska|  Kowalski|     23.07.2003|       10254|  556,62 z|
|Polska|     Dudek|     15.07.2003|       10255|2 490,50 z|
|Niemcy| Leverling|     17.07.2003|       10256|  517,80 z|
|Niemcy|   Peacock|     22.07.2003|       10257|1 119,90 z|
|Niemcy|   Davolio|     23.07.2003|       10258|1 614,88 z|
|Niemcy|   Peacock|     25.07.2003|       10259|  100,80 z|
|Niemcy|   Peacock|     29.07.2003|       10260|1 504,65 z|
|Niemcy|   Peacock|     

In [5]:
unique_sprzedawcy = df.select("Sprzedawca").distinct()
unique_sprzedawcy.show()

+----------+
|Sprzedawca|
+----------+
|  Sowiński|
|   Peacock|
|      King|
|     Dudek|
|   Davolio|
|    Fuller|
| Leverling|
|  Kowalski|
|  Callahan|
+----------+



In [6]:
from pyspark.sql.functions import to_date

df = df.withColumn("Data zamowienia", to_date(col("Data zamowienia"), "dd.MM.yyyy"))

df.show()

+------+----------+---------------+------------+-----------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia|      Utarg|
+------+----------+---------------+------------+-----------+
|Polska|  Kowalski|     2003-07-16|       10248|  440,00 z|
|Polska|  Sowiński|     2003-07-10|       10249|1 863,40 z|
|Niemcy|   Peacock|     2003-07-12|       10250|1 552,60 z|
|Niemcy| Leverling|     2003-07-15|       10251|  654,06 z|
|Niemcy|   Peacock|     2003-07-11|       10252|3 597,90 z|
|Niemcy| Leverling|     2003-07-16|       10253|1 444,80 z|
|Polska|  Kowalski|     2003-07-23|       10254|  556,62 z|
|Polska|     Dudek|     2003-07-15|       10255|2 490,50 z|
|Niemcy| Leverling|     2003-07-17|       10256|  517,80 z|
|Niemcy|   Peacock|     2003-07-22|       10257|1 119,90 z|
|Niemcy|   Davolio|     2003-07-23|       10258|1 614,88 z|
|Niemcy|   Peacock|     2003-07-25|       10259|  100,80 z|
|Niemcy|   Peacock|     2003-07-29|       10260|1 504,65 z|
|Niemcy|   Peacock|     

In [7]:
df = df.withColumn("Utarg", regexp_replace(col("Utarg"), "[ z|]", ""))
df = df.withColumn("Utarg", regexp_replace(col("Utarg"), ",", ".").cast(FloatType()))

df.show()

+------+----------+---------------+------------+-------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia|  Utarg|
+------+----------+---------------+------------+-------+
|Polska|  Kowalski|     2003-07-16|       10248|  440.0|
|Polska|  Sowiński|     2003-07-10|       10249| 1863.4|
|Niemcy|   Peacock|     2003-07-12|       10250| 1552.6|
|Niemcy| Leverling|     2003-07-15|       10251| 654.06|
|Niemcy|   Peacock|     2003-07-11|       10252| 3597.9|
|Niemcy| Leverling|     2003-07-16|       10253| 1444.8|
|Polska|  Kowalski|     2003-07-23|       10254| 556.62|
|Polska|     Dudek|     2003-07-15|       10255| 2490.5|
|Niemcy| Leverling|     2003-07-17|       10256|  517.8|
|Niemcy|   Peacock|     2003-07-22|       10257| 1119.9|
|Niemcy|   Davolio|     2003-07-23|       10258|1614.88|
|Niemcy|   Peacock|     2003-07-25|       10259|  100.8|
|Niemcy|   Peacock|     2003-07-29|       10260|1504.65|
|Niemcy|   Peacock|     2003-07-30|       10261|  448.0|
|Niemcy|  Callahan|     2003-07

In [8]:
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumnRenamed("idZamowienia", "idZamowienia_temp") \
       .withColumn("idZamowienia", col("idZamowienia_temp").cast(IntegerType())) \
       .drop("idZamowienia_temp") \
       .repartition(1).withColumn('ID', monotonically_increasing_id())

df.show()

+------+----------+---------------+-------+------------+---+
|  Kraj|Sprzedawca|Data zamowienia|  Utarg|idZamowienia| ID|
+------+----------+---------------+-------+------------+---+
|Polska|  Kowalski|     2003-07-16|  440.0|       10248|  0|
|Polska|  Sowiński|     2003-07-10| 1863.4|       10249|  1|
|Niemcy|   Peacock|     2003-07-12| 1552.6|       10250|  2|
|Niemcy| Leverling|     2003-07-15| 654.06|       10251|  3|
|Niemcy|   Peacock|     2003-07-11| 3597.9|       10252|  4|
|Niemcy| Leverling|     2003-07-16| 1444.8|       10253|  5|
|Polska|  Kowalski|     2003-07-23| 556.62|       10254|  6|
|Polska|     Dudek|     2003-07-15| 2490.5|       10255|  7|
|Niemcy| Leverling|     2003-07-17|  517.8|       10256|  8|
|Niemcy|   Peacock|     2003-07-22| 1119.9|       10257|  9|
|Niemcy|   Davolio|     2003-07-23|1614.88|       10258| 10|
|Niemcy|   Peacock|     2003-07-25|  100.8|       10259| 11|
|Niemcy|   Peacock|     2003-07-29|1504.65|       10260| 12|
|Niemcy|   Peacock|     

In [9]:
columns = df.columns

new_column_order = ["ID"] + [col for col in columns if col != "ID"]

df = df.select(new_column_order)

df.show()

+---+------+----------+---------------+-------+------------+
| ID|  Kraj|Sprzedawca|Data zamowienia|  Utarg|idZamowienia|
+---+------+----------+---------------+-------+------------+
|  0|Polska|  Kowalski|     2003-07-16|  440.0|       10248|
|  1|Polska|  Sowiński|     2003-07-10| 1863.4|       10249|
|  2|Niemcy|   Peacock|     2003-07-12| 1552.6|       10250|
|  3|Niemcy| Leverling|     2003-07-15| 654.06|       10251|
|  4|Niemcy|   Peacock|     2003-07-11| 3597.9|       10252|
|  5|Niemcy| Leverling|     2003-07-16| 1444.8|       10253|
|  6|Polska|  Kowalski|     2003-07-23| 556.62|       10254|
|  7|Polska|     Dudek|     2003-07-15| 2490.5|       10255|
|  8|Niemcy| Leverling|     2003-07-17|  517.8|       10256|
|  9|Niemcy|   Peacock|     2003-07-22| 1119.9|       10257|
| 10|Niemcy|   Davolio|     2003-07-23|1614.88|       10258|
| 11|Niemcy|   Peacock|     2003-07-25|  100.8|       10259|
| 12|Niemcy|   Peacock|     2003-07-29|1504.65|       10260|
| 13|Niemcy|   Peacock| 

In [10]:
df.printSchema()

root
 |-- ID: long (nullable = false)
 |-- Kraj: string (nullable = true)
 |-- Sprzedawca: string (nullable = true)
 |-- Data zamowienia: date (nullable = true)
 |-- Utarg: float (nullable = true)
 |-- idZamowienia: integer (nullable = true)



## Zadanie 2

In [11]:
from pyspark.sql import functions as F
import time

bucketed_df = df.write \
    .bucketBy(4, "Kraj") \
    .sortBy("Kraj") \
    .mode("overwrite") \
    .saveAsTable("bucketed_orders2")

bucketed_df = spark.table("bucketed_orders2")

start_time = time.time()
bucketed_aggregated = bucketed_df.groupBy("Kraj").agg(
    F.sum("Utarg").alias("Suma_Utargu"),
    F.count("idZamowienia").alias("Liczba_zamowien")
)
bucketed_aggregated.show()
end_time = time.time()
bucketed_time = end_time - start_time
print(f"Czas wykonania zapytania na danych z wiaderkami: {bucketed_time} sekund")

start_time = time.time()
original_aggregated = df.groupBy("Kraj").agg(
    F.sum("Utarg").alias("Suma_Utargu"),
    F.count("idZamowienia").alias("Liczba_zamowien")
)
original_aggregated.show()
end_time = time.time()
original_time = end_time - start_time
print(f"Czas wykonania zapytania na danych oryginalnych: {original_time} sekund")


+------+-----------------+---------------+
|  Kraj|      Suma_Utargu|Liczba_zamowien|
+------+-----------------+---------------+
|Polska| 333330.908657074|            215|
|Niemcy|894996.4916362762|            584|
+------+-----------------+---------------+

Czas wykonania zapytania na danych z wiaderkami: 0.22080278396606445 sekund
+------+-----------------+---------------+
|  Kraj|      Suma_Utargu|Liczba_zamowien|
+------+-----------------+---------------+
|Polska| 333330.908657074|            215|
|Niemcy|894996.4916362762|            584|
+------+-----------------+---------------+

Czas wykonania zapytania na danych oryginalnych: 0.16924571990966797 sekund


In [12]:
df.write \
    .partitionBy("Kraj") \
    .mode("overwrite") \
    .csv("partitioned_by_kraj")

df.write \
    .partitionBy("Sprzedawca") \
    .mode("overwrite") \
    .csv("partitioned_by_sprzedawca")


In [28]:
import pyspark.sql.functions as F
import time

start_time = time.time()
df_aggregated = df.groupBy("Kraj").agg(
    sum("Utarg").alias("Suma_Utargu"),
    count("idZamowienia").alias("Liczba_zamowien")
)
df_aggregated.show()
end_time = time.time()
original_time = end_time - start_time
print(f"Czas wykonania zapytania na danych oryginalnych: {original_time} sekund")

df_partitioned = df.repartition("Kraj")

start_time = time.time()
df_partitioned_aggregated = df_partitioned.groupBy("Kraj").agg(
    F.sum("Utarg").alias("Suma_Utargu"),
    F.count("idZamowienia").alias("Liczba_zamowien")
)
df_partitioned_aggregated.show()
end_time = time.time()
partitioned_time = end_time - start_time
print(f"Czas wykonania zapytania na danych partycjonowanych: {partitioned_time} sekund")


+------+-----------------+---------------+
|  Kraj|      Suma_Utargu|Liczba_zamowien|
+------+-----------------+---------------+
|Polska| 333330.908657074|            215|
|Niemcy|894996.4916362762|            584|
+------+-----------------+---------------+

Czas wykonania zapytania na danych oryginalnych: 0.09369325637817383 sekund
+------+-----------------+---------------+
|  Kraj|      Suma_Utargu|Liczba_zamowien|
+------+-----------------+---------------+
|Niemcy|894996.4916362762|            584|
|Polska| 333330.908657074|            215|
+------+-----------------+---------------+

Czas wykonania zapytania na danych partycjonowanych: 0.06855225563049316 sekund


# Zadanie 3

In [48]:
subset_1 = df.sample(withReplacement=False, fraction=0.5, seed=42) \
    .withColumn("month", F.month(col("Data zamowienia")))
subset_1.show(6)

+---+------+----------+---------------+-------+------------+-----+
| ID|  Kraj|Sprzedawca|Data zamowienia|  Utarg|idZamowienia|month|
+---+------+----------+---------------+-------+------------+-----+
|  3|Niemcy| Leverling|     2003-07-15| 654.06|       10251|    7|
|  7|Polska|     Dudek|     2003-07-15| 2490.5|       10255|    7|
| 10|Niemcy|   Davolio|     2003-07-23|1614.88|       10258|    7|
| 12|Niemcy|   Peacock|     2003-07-29|1504.65|       10260|    7|
| 16|Polska|  Sowiński|     2003-08-23| 695.62|       10264|    8|
| 18|Niemcy| Leverling|     2003-07-31| 346.56|       10266|    7|
+---+------+----------+---------------+-------+------------+-----+
only showing top 6 rows



In [57]:
subset_2 = df.sample(withReplacement=False, fraction=0.5, seed=42) \
    .withColumn("net_value", (col("Utarg").cast("float") / 1.23).cast("float"))

subset_2.show(5)

+---+------+----------+---------------+-------+------------+---------+
| ID|  Kraj|Sprzedawca|Data zamowienia|  Utarg|idZamowienia|net_value|
+---+------+----------+---------------+-------+------------+---------+
|  3|Niemcy| Leverling|     2003-07-15| 654.06|       10251| 531.7561|
|  7|Polska|     Dudek|     2003-07-15| 2490.5|       10255|2024.7968|
| 10|Niemcy|   Davolio|     2003-07-23|1614.88|       10258|1312.9105|
| 12|Niemcy|   Peacock|     2003-07-29|1504.65|       10260|1223.2927|
| 16|Polska|  Sowiński|     2003-08-23| 695.62|       10264|565.54474|
+---+------+----------+---------------+-------+------------+---------+
only showing top 5 rows



In [69]:
subset_3 = df.sample(withReplacement=False, fraction=0.5, seed=42) \
    .withColumn("Sprzedawca_upper", F.upper(col("Sprzedawca")))

subset_3.show(10)

+---+------+----------+---------------+-------+------------+----------------+
| ID|  Kraj|Sprzedawca|Data zamowienia|  Utarg|idZamowienia|Sprzedawca_upper|
+---+------+----------+---------------+-------+------------+----------------+
|  3|Niemcy| Leverling|     2003-07-15| 654.06|       10251|       LEVERLING|
|  7|Polska|     Dudek|     2003-07-15| 2490.5|       10255|           DUDEK|
| 10|Niemcy|   Davolio|     2003-07-23|1614.88|       10258|         DAVOLIO|
| 12|Niemcy|   Peacock|     2003-07-29|1504.65|       10260|         PEACOCK|
| 16|Polska|  Sowiński|     2003-08-23| 695.62|       10264|        SOWIŃSKI|
| 18|Niemcy| Leverling|     2003-07-31| 346.56|       10266|       LEVERLING|
| 20|Niemcy|  Callahan|     2003-08-02| 1101.2|       10268|        CALLAHAN|
| 25|Niemcy| Leverling|     2003-08-12|2037.28|       10273|       LEVERLING|
| 26|Polska|  Sowiński|     2003-08-16|  538.6|       10274|        SOWIŃSKI|
| 27|Niemcy|   Davolio|     2003-08-09| 291.84|       10275|    

In [70]:
subset_4 = df.sample(withReplacement=False, fraction=0.5, seed=42) \
    .withColumn("currency", F.lit("PLN"))

subset_4.show(5)

+---+------+----------+---------------+-------+------------+--------+
| ID|  Kraj|Sprzedawca|Data zamowienia|  Utarg|idZamowienia|currency|
+---+------+----------+---------------+-------+------------+--------+
|  3|Niemcy| Leverling|     2003-07-15| 654.06|       10251|     PLN|
|  7|Polska|     Dudek|     2003-07-15| 2490.5|       10255|     PLN|
| 10|Niemcy|   Davolio|     2003-07-23|1614.88|       10258|     PLN|
| 12|Niemcy|   Peacock|     2003-07-29|1504.65|       10260|     PLN|
| 16|Polska|  Sowiński|     2003-08-23| 695.62|       10264|     PLN|
+---+------+----------+---------------+-------+------------+--------+
only showing top 5 rows



In [71]:
subset_1.createOrReplaceTempView("subset_1_table")
subset_2.write.mode("overwrite").parquet("subset_2.parquet")
subset_3.write.mode("overwrite").csv("subset_3.csv", header=True)
subset_4.write.mode("overwrite").json("subset_4.json")


In [72]:
subset_2_from_file = spark.read.parquet("subset_2.parquet")
subset_3_from_file = spark.read.csv("subset_3.csv", header=True, inferSchema=True)
subset_4_from_file = spark.read.json("subset_4.json")


subset_2_from_file.createOrReplaceTempView("subset_2_table")
subset_3_from_file.createOrReplaceTempView("subset_3_table")
subset_4_from_file.createOrReplaceTempView("subset_4_table")


In [73]:
result = spark.sql("""
SELECT 
    s1.idZamowienia, 
    s1.Kraj, 
    s3.Sprzedawca, 
    s1.`Data zamowienia`, 
    s1.Utarg,
    s1.month, 
    s2.net_value, 
    s4.currency,
    s3.Sprzedawca_upper
FROM 
    subset_1_table s1
LEFT JOIN 
    subset_2_table s2 ON s1.idZamowienia = s2.idZamowienia
LEFT JOIN 
    subset_3_table s3 ON s1.idZamowienia = s3.idZamowienia
LEFT JOIN 
    subset_4_table s4 ON s1.idZamowienia = s4.idZamowienia
""")
result.show()

+------------+------+----------+---------------+-------+-----+---------+--------+----------------+
|idZamowienia|  Kraj|Sprzedawca|Data zamowienia|  Utarg|month|net_value|currency|Sprzedawca_upper|
+------------+------+----------+---------------+-------+-----+---------+--------+----------------+
|       10251|Niemcy| Leverling|     2003-07-15| 654.06|    7| 531.7561|     PLN|       LEVERLING|
|       10255|Polska|     Dudek|     2003-07-15| 2490.5|    7|2024.7968|     PLN|           DUDEK|
|       10258|Niemcy|   Davolio|     2003-07-23|1614.88|    7|1312.9105|     PLN|         DAVOLIO|
|       10260|Niemcy|   Peacock|     2003-07-29|1504.65|    7|1223.2927|     PLN|         PEACOCK|
|       10264|Polska|  Sowiński|     2003-08-23| 695.62|    8|565.54474|     PLN|        SOWIŃSKI|
|       10266|Niemcy| Leverling|     2003-07-31| 346.56|    7| 281.7561|     PLN|       LEVERLING|
|       10268|Niemcy|  Callahan|     2003-08-02| 1101.2|    8| 895.2845|     PLN|        CALLAHAN|
|       10